In [1]:
import sys

sys.path.append("/Users/j/Documents/Forme/code/forme-groups")

from src.groups.unit import Unit, UnitType, UnitTypeRef, UnitTypeFunction
from src.groups.group import Group, Data


def main():
    # Create a default group
    group = Group(
        system_types_path="/Users/j/Documents/Forme/code/forme-groups/src/groups/system_types.json"
    )

    # Create function to be used as a system function
    def test_function(input_str):
        if input_str.startswith("test<") and input_str.endswith(">") and "|" in input_str:
            return input_str[5:-1].split("|")
        elif input_str.startswith("test<") and input_str.endswith(">"):
            return input_str[5:-1]
        else:
            return input_str

    # Create a new unit type
    new_unit_type = UnitType(
        aliases= (UnitTypeRef("test_unit_type"),),
        super_type=UnitTypeRef("str"),
        prefix="test<",
        suffix=">",
        separator="|",
        sys_function=UnitTypeFunction(
            object=test_function,
            args=(),
        )
    )

    # Add a unit type to the group
    group._group_unit_generator.unit_generator.unit_type_pool.add_unit_type(
        new_unit_type
    )

    # Freeze the unit types
    group.freeze_unit_types()
    print(group._group_unit_generator.unit_generator.unit_type_pool.get_type_aliases())
    print(group.group_units)
    
    # Create a new unit - Nonce: 0
    unit1 = group.create_group_unit()
    print(unit1)

    # Create another new unit - Nonce: 1
    group.create_group_unit()
    print(group.get_nonce_tiers())
    print(group.group_units)

    # Create a new unit with data - Nonce: 2
    data_unit: Unit = group._group_unit_generator.unit_generator.create_unit(
        alias="test_unit_type",
        value="test<test1|test2|test3>"
    )
    data_object: Data = group._group_unit_generator.create_data(data=(data_unit,))
    new_unit = group.create_group_unit(data=data_object)
    print(new_unit)
    print(new_unit.data.is_schema())
    print(group._group_unit_generator.unit_generator.format_unit(new_unit.data.entries[0]))
    print(group.format_group_unit(new_unit))
    #print(group.format_group_unit())

    # Create a new unit with a schema - Nonce: 3
    schema_unit: Unit = group._group_unit_generator.unit_generator.create_unit(
        alias="dict",
        value={'Schema': {'test': 'test'}}
    )
    schema_object: Data = group._group_unit_generator.create_data(data=(schema_unit,))
    new_unit = group.create_group_unit(data=schema_object)
    print(new_unit)
    print(new_unit.data.is_schema())

main()

['string', 'str', 'int', 'integer', 'float', 'bool', 'boolean', 'list', 'dict', 'dictionary', 'tuple', 'bytes', 'test_unit_type']
[]
Creating new group unit with default nonce.
GroupUnit(nonce=0, ownership=None, credentials=None, data=None)
Creating new group unit from previous nonce.
1
[GroupUnit(nonce=0, ownership=None, credentials=None, data=None), GroupUnit(nonce=1, ownership=None, credentials=None, data=None)]
Creating new group unit from previous nonce.
GroupUnit(nonce=2, ownership=None, credentials=None, data=Data(entries=(Unit(value=['test1', 'test2', 'test3'], type_ref=UnitTypeRef(alias='test_unit_type')),)))
False
test<test1|test2|test3>
Nonce: 2
Ownership: 
Credentials: 
Data: test<test1|test2|test3>
Creating new group unit from previous nonce.
GroupUnit(nonce=3, ownership=None, credentials=None, data=Data(entries=(Unit(value={'Schema': {'test': 'test'}}, type_ref=UnitTypeRef(alias='dict')),)))
True
